In [1]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 1.4 MB/s 


In [2]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf
 教授资格证身份证.pdf


In [4]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [5]:
!ls

 a0529-a0611_UK     dataframe_csv_UK
 a0529-a0611_US     dataframe_result_529_UK.csv
 a0529.csv	    dataframe_result_530_UK.csv
 a0530.csv	    dataframe_result_531_UK.csv
 a0531.csv	    dataframe_result_csv
 a0601.csv	    dataframe_result_csv_EU
 a0602.csv	    dataframe_result_csv_UK
 a0603.csv	    Github
 a0604.csv	    heatmap529_UK.ipynb
 a0605.csv	    heatmap530.ipynb
 a0606.csv	    Heat_Map_Online_language.ipynb
 a0607.csv	    __MACOSX
 a0608.csv	    mapping_UK
 a0610.csv	    mapping_US
 a0611.csv	    non_violant1.csv
 ccc_compiled.csv   non_violant2.csv
 csv		    UK
 csv.zip	    violant.csv
 dataframe_csv	   '新建 Microsoft PowerPoint 演示文稿.pptx'
 dataframe_csv_EU


In [6]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
#from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [7]:
ct=pd.read_csv('a0529-a0611_UK/a0531.csv',encoding='ISO-8859-1')

In [8]:
!ls

 a0529-a0611_UK     dataframe_csv_UK
 a0529-a0611_US     dataframe_result_529_UK.csv
 a0529.csv	    dataframe_result_530_UK.csv
 a0530.csv	    dataframe_result_531_UK.csv
 a0531.csv	    dataframe_result_csv
 a0601.csv	    dataframe_result_csv_EU
 a0602.csv	    dataframe_result_csv_UK
 a0603.csv	    Github
 a0604.csv	    heatmap529_UK.ipynb
 a0605.csv	    heatmap530.ipynb
 a0606.csv	    Heat_Map_Online_language.ipynb
 a0607.csv	    __MACOSX
 a0608.csv	    mapping_UK
 a0610.csv	    mapping_US
 a0611.csv	    non_violant1.csv
 ccc_compiled.csv   non_violant2.csv
 csv		    UK
 csv.zip	    violant.csv
 dataframe_csv	   '新建 Microsoft PowerPoint 演示文稿.pptx'
 dataframe_csv_EU


In [9]:
###
#ct=pd.read_csv('a0601.csv')

In [10]:
ct

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title
0,0,52552983299_10157916498178300,Facebook,2020-05-31 23:59:56,2021-08-31 12:54:55,link,George Floyd Death Protesters Return To Capito...,denver.cbslocal.com,"Smart: ""We stand for the truth. We stand for j...",Protesters returned to the Colorado State Capi...,1.0,https://denver.cbslocal.com/2020/05/31/george-...,https://www.facebook.com/52552983299/posts/101...,502230,1.463158,1.0,34,7,41,15,2,2,7,31,0,0,19,8,22,12,5,7,9,13,0,4,20339,CBS Denver,CBSDenver,https://scontent-den4-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Covering Colorado First. Your #1 source for br...,True,2009-02-25 23:34:30,en,0,20339|10157916498178300,{'Denver': 1},{},{'Denver': 1}
1,1,57427307078_10159355042537079,Facebook,2020-05-31 23:59:53,2021-08-31 20:39:39,link,Psychology professor explains how race factore...,abc7news.com,"Smart: ""We stand for the truth. We stand for j...",What started out as a mostly peaceful demonstr...,1.0,https://abc7news.com/6222900/?ex_cid=TA_KGO_FB...,https://www.facebook.com/57427307078/posts/101...,1219608,-5.451613,1.0,14,5,6,1,0,2,0,3,0,0,50,15,41,9,9,10,21,14,0,6,19637,ABC7 News,abc7news,https://scontent-den4-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Get breaking news and the most talked about st...,True,2009-01-26 18:11:53,en,0,19637|10159355042537079,{'Los Angeles': 1},{},{'Los Angeles': 1}
2,2,129617873765147_3588816481178585,Facebook,2020-05-31 23:59:53,2021-06-20 22:23:37,link,Truck hurtles towards protestors in Minneapoli...,express.co.uk,A SHOCKING video has shown a tanker truck driv...,Follow our latest updates on the riots as tens...,1.0,https://www.express.co.uk/news/world/1289602/g...,https://www.facebook.com/129617873765147/posts...,1768821,-3.893333,1.0,17,8,34,0,1,2,2,11,0,0,96,14,105,15,5,29,8,20,0,6,89685,Daily Express,DailyExpress,https://scontent-den4-1.xx.fbcdn.net/v/t1.6435...,facebook_page,GB,The Official Facebook page of the Daily and Su...,True,2010-12-10 10:07:29,en,0,89685|3588816481178585,{},"{'Minneapolis': 1, 'US': 1}",{'Minneapolis': 1}
3,3,821532787927852_3008252662589176,Facebook,2020-05-31 23:59:52,2021-07-09 03:56:24,native_video,Truck hurtles towards protestors in Minneapoli...,express.co.uk,A SHOCKING video has shown a tanker truck driv...,Are You a Uniter or a Divider - Bible lesson T...,21.0,https://www.facebook.com/GunGuyTV/videos/25336...,https://www.facebook.com/821532787927852/posts...,4813,-2.000000,2.0,11,3,0,0,0,0,0,0,0,0,10,5,2,4,1,2,1,3,0,1,11480437,Gunguytv,GunGuyTV,https://scontent-den4-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Find out more about the GunGuy at https://joel...,False,2015-05-01 05:50:26,en,0,11480437|3008252662589176,{},"{'Minneapolis': 1, 'US': 1}",{'Minneapolis': 1}
4,4,123264117724844_3292223030828921,Facebook,2020-05-31 23:59:49,2021-06-27 10:36:41,photo,Truck hurtles towards protestors in Minneapoli...,express.co.uk,A SHOCKING video has shown a tanker truck driv...,Trumbull officials issue statement on George F...,2.0,https://www.facebook.com/TrumbullTimes/photos/...,https://www.facebook.com/123264117724844/posts...,3621,1.695652,1.0,30,2,0,7,0,0,0,0,0,0,6,3,5,6,1,0,2,0,0,0,28309,The Trumbull Times,TrumbullTimes,https://scontent-den4-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,"Your leader in Trumbull news, sports

In [11]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [12]:
#ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [13]:
###
ct_ner = ct

In [14]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,{'Denver': 1},{},{'Denver': 1}
1,{'Los Angeles': 1},{},{'Los Angeles': 1}
2,{},"{'Minneapolis': 1, 'US': 1}",{'Minneapolis': 1}
3,{},"{'Minneapolis': 1, 'US': 1}",{'Minneapolis': 1}
4,{},"{'Minneapolis': 1, 'US': 1}",{'Minneapolis': 1}


In [15]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

In [16]:
ct_ner['NER-msg']

0                        [Denver]
1                   [Los Angeles]
2                              []
3                              []
4                              []
                   ...           
39015                          []
39016                          []
39017    [Los Angeles City, L.A.]
39018                          []
39019                          []
Name: NER-msg, Length: 39020, dtype: object

In [17]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

In [18]:
ct_ner['NER']

0                                         [Denver, Denver]
1                               [Los Angeles, Los Angeles]
2                           [Minneapolis, US, Minneapolis]
3                           [Minneapolis, US, Minneapolis]
4                           [Minneapolis, US, Minneapolis]
                               ...                        
39015                                        [Los Angeles]
39016                                                   []
39017    [Los Angeles City, L.A., Minneapolis, Fairfax,...
39018                              [Portland, Minneapolis]
39019                                        [Los Angeles]
Name: NER, Length: 39020, dtype: object

In [19]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

In [20]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

In [21]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER
0,0,52552983299_10157916498178300,Facebook,2020-05-31 23:59:56,2021-08-31 12:54:55,link,George Floyd Death Protesters Return To Capito...,denver.cbslocal.com,"Smart: ""We stand for the truth. We stand for j...",Protesters returned to the Colorado State Capi...,1.0,https://denver.cbslocal.com/2020/05/31/george-...,https://www.facebook.com/52552983299/posts/101...,502230,1.463158,1.0,34,7,41,15,2,2,7,31,0,0,19,8,22,12,5,7,9,13,0,4,20339,CBS Denver,CBSDenver,https://scontent-den4-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Covering Colorado First. Your #1 source for br...,True,2009-02-25 23:34:30,en,0,20339|10157916498178300,[Denver],[],[Denver],[denver]
1,1,57427307078_10159355042537079,Facebook,2020-05-31 23:59:53,2021-08-31 20:39:39,link,Psychology professor explains how race factore...,abc7news.com,"Smart: ""We stand for the truth. We stand for j...",What started out as a mostly peaceful demonstr...,1.0,https://abc7news.com/6222900/?ex_cid=TA_KGO_FB...,https://www.facebook.com/57427307078/posts/101...,1219608,-5.451613,1.0,14,5,6,1,0,2,0,3,0,0,50,15,41,9,9,10,21,14,0,6,19637,ABC7 News,abc7news,https://scontent-den4-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Get breaking news and the most talked about st...,True,2009-01-26 18:11:53,en,0,19637|10159355042537079,[Los Angeles],[],[Los Angeles],[los angeles]
2,2,129617873765147_3588816481178585,Facebook,2020-05-31 23:59:53,2021-06-20 22:23:37,link,Truck hurtles towards protestors in Minneapoli...,express.co.uk,A SHOCKING video has shown a tanker truck driv...,Follow our latest updates on the riots as tens...,1.0,https://www.express.co.uk/news/world/1289602/g...,https://www.facebook.com/129617873765147/posts...,1768821,-3.893333,1.0,17,8,34,0,1,2,2,11,0,0,96,14,105,15,5,29,8,20,0,6,89685,Daily Express,DailyExpress,https://scontent-den4-1.xx.fbcdn.net/v/t1.6435...,facebook_page,GB,The Official Facebook page of the Daily and Su...,True,2010-12-10 10:07:29,en,0,89685|3588816481178585,[],"[Minneapolis, US]",[Minneapolis],"[minneapolis, us]"
3,3,821532787927852_3008252662589176,Facebook,2020-05-31 23:59:52,2021-07-09 03:56:24,native_video,Truck hurtles towards protestors in Minneapoli...,express.co.uk,A SHOCKING video has shown a tanker truck driv...,Are You a Uniter or a Divider - Bible lesson T...,21.0,https://www.facebook.com/GunGuyTV/videos/25336...,https://www.facebook.com/821532787927852/posts...,4813,-2.000000,2.0,11,3,0,0,0,0,0,0,0,0,10,5,2,4,1,2,1,3,0,1,11480437,Gunguytv,GunGuyTV,https://scontent-den4-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Find out more about the GunGuy at https://joel...,False,2015-05-01 05:50:26,en,0,11480437|3008252662589176,[],"[Minneapolis, US]",[Minneapolis],"[minneapolis, us]"
4,4,123264117724844_3292223030828921,Facebook,2020-05-31 23:59:49,2021-06-27 10:36:41,photo,Truck hurtles towards protestors in Minneapoli...,express.co.uk,A SHOCKING video has shown a tanker truck driv...,Trumbull officials issue statement on George F...,2.0,https://www.facebook.com/TrumbullTimes/photos/...,https://www.facebook.com/123264117724844/posts...,3621,1.695652,1.0,30,2,0,7,0,0,0,0,0,0,6,3,5,6,1,0,2,0,0,0,28309,The Trumbull Times,TrumbullTimes,https://scontent-den4-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,"Your leader in Trum

In [22]:
ct_ner['NER-1']=ct_ner['NER']

In [23]:
ct_ner['NER-1'][1]

['los angeles']

In [24]:
!pip install -q geopy
!pip install -q pyproj

# basic utilities
import json
import requests
import pandas as pd
import pprint
import time

# reverse geocoding utilities
from geopy.geocoders import Nominatim

# geography projection utilities
import pyproj
from shapely.ops import transform
from shapely.geometry import Point
from functools import partial

     |████████████████████████████████| 6.3 MB 4.2 MB/s 


In [25]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
#from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [26]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,52552983299_10157916498178300,Facebook,2020-05-31 23:59:56,2021-08-31 12:54:55,link,George Floyd Death Protesters Return To Capito...,denver.cbslocal.com,"Smart: ""We stand for the truth. We stand for j...",Protesters returned to the Colorado State Capi...,1.0,https://denver.cbslocal.com/2020/05/31/george-...,https://www.facebook.com/52552983299/posts/101...,502230,1.463158,1.0,34,7,41,15,2,2,7,31,0,0,19,8,22,12,5,7,9,13,0,4,20339,CBS Denver,CBSDenver,https://scontent-den4-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Covering Colorado First. Your #1 source for br...,True,2009-02-25 23:34:30,en,0,20339|10157916498178300,[Denver],[],[Denver],[denver],[denver]
1,1,57427307078_10159355042537079,Facebook,2020-05-31 23:59:53,2021-08-31 20:39:39,link,Psychology professor explains how race factore...,abc7news.com,"Smart: ""We stand for the truth. We stand for j...",What started out as a mostly peaceful demonstr...,1.0,https://abc7news.com/6222900/?ex_cid=TA_KGO_FB...,https://www.facebook.com/57427307078/posts/101...,1219608,-5.451613,1.0,14,5,6,1,0,2,0,3,0,0,50,15,41,9,9,10,21,14,0,6,19637,ABC7 News,abc7news,https://scontent-den4-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Get breaking news and the most talked about st...,True,2009-01-26 18:11:53,en,0,19637|10159355042537079,[Los Angeles],[],[Los Angeles],[los angeles],[los angeles]
2,2,129617873765147_3588816481178585,Facebook,2020-05-31 23:59:53,2021-06-20 22:23:37,link,Truck hurtles towards protestors in Minneapoli...,express.co.uk,A SHOCKING video has shown a tanker truck driv...,Follow our latest updates on the riots as tens...,1.0,https://www.express.co.uk/news/world/1289602/g...,https://www.facebook.com/129617873765147/posts...,1768821,-3.893333,1.0,17,8,34,0,1,2,2,11,0,0,96,14,105,15,5,29,8,20,0,6,89685,Daily Express,DailyExpress,https://scontent-den4-1.xx.fbcdn.net/v/t1.6435...,facebook_page,GB,The Official Facebook page of the Daily and Su...,True,2010-12-10 10:07:29,en,0,89685|3588816481178585,[],"[Minneapolis, US]",[Minneapolis],"[minneapolis, us]","[minneapolis, us]"
3,3,821532787927852_3008252662589176,Facebook,2020-05-31 23:59:52,2021-07-09 03:56:24,native_video,Truck hurtles towards protestors in Minneapoli...,express.co.uk,A SHOCKING video has shown a tanker truck driv...,Are You a Uniter or a Divider - Bible lesson T...,21.0,https://www.facebook.com/GunGuyTV/videos/25336...,https://www.facebook.com/821532787927852/posts...,4813,-2.000000,2.0,11,3,0,0,0,0,0,0,0,0,10,5,2,4,1,2,1,3,0,1,11480437,Gunguytv,GunGuyTV,https://scontent-den4-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Find out more about the GunGuy at https://joel...,False,2015-05-01 05:50:26,en,0,11480437|3008252662589176,[],"[Minneapolis, US]",[Minneapolis],"[minneapolis, us]","[minneapolis, us]"
4,4,123264117724844_3292223030828921,Facebook,2020-05-31 23:59:49,2021-06-27 10:36:41,photo,Truck hurtles towards protestors in Minneapoli...,express.co.uk,A SHOCKING video has shown a tanker truck driv...,Trumbull officials issue statement on George F...,2.0,https://www.facebook.com/TrumbullTimes/photos/...,https://www.facebook.com/123264117724844/posts...,3621,1.695652,1.0,30,2,0,7,0,0,0,0,0,0,6,3,5,6,1,0,2,0,0,0,28309,The Trumbull Times,TrumbullTimes,https://scontent-d

In [27]:
%cd mapping_UK

/content/gdrive/My Drive/2021-online-language-offline-impact-main/mapping_UK


In [28]:
!ls

Counties_UK1.txt	       dataframe_mapping_0605.csv
Counties_UK.txt		       dataframe_mapping_0606.csv
Country.txt		       dataframe_mapping_0607.csv
dataframe0529_post_remove.csv  dataframe_mapping_0608.csv
dataframe0610_post_remove.csv  dataframe_mapping_0609.csv
dataframe_mapping	       dataframe_mapping_0610.csv
dataframe_mapping_0529.csv     dataframe_mapping_0611.csv
dataframe_mapping_0530.csv     dataframe_mapping_remove_0529.csv
dataframe_mapping_0531.csv     dataframe_mapping_remove_0606.csv
dataframe_mapping_0601.csv     dataframe_post
dataframe_mapping_0602.csv     Post-specific_UK.zip
dataframe_mapping_0603.csv     states.csv
dataframe_mapping_0604.csv


In [29]:
ct_mapping=pd.read_csv('dataframe_mapping_0531.csv',encoding='ISO-8859-1')

In [30]:
len(ct_mapping['City'])

3788

In [31]:
country_list=pd.read_csv('Country.txt')
country_list['Name']=country_list['Name'].str.lower()

In [32]:
country_list['Name']

0            afghanistan
1          �land islands
2                albania
3                algeria
4         american samoa
             ...        
250                   us
251    the united states
252              america
253                  u.s
254        united states
Name: Name, Length: 255, dtype: object

In [33]:
states_list=pd.read_csv('states.csv')
states_list['State']

0                             Alabama, United States
1                              Alaska, United States
2                             Arizona, United States
3                            Arkansas, United States
4                          California, United States
                           ...                      
165         Armagh, Northern Ireland, United Kingdom
166           Down, Northern Ireland, United Kingdom
167      Fermanagh, Northern Ireland, United Kingdom
168    Londonderry, Northern Ireland, United Kingdom
169         Tyrone, Northern Ireland, United Kingdom
Name: State, Length: 170, dtype: object

In [34]:
from numpy.core.numeric import NaN
for i in range(len(ct_mapping['City'])):
  for k in range(len(country_list['Name'])):
    if ct_mapping['City'][i]==country_list['Name'][k]:
      ct_mapping['Mapping'][i]=NaN

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [35]:
for i in range(len(ct_mapping['City'])):
  for k in range(len(states_list['State'])):
    if ct_mapping['Mapping'][i]==states_list['State'][k]:
      ct_mapping['Mapping'][i]=NaN

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [36]:
for i in range(len(ct_mapping['City'])):
  for k in range(len(country_list['Name'])):
    if ct_mapping['Mapping'][i]==country_list['Name'][k]:
      ct_mapping['Mapping'][i]=NaN

In [37]:
ct_mapping.to_csv('dataframe_mapping_remove_0531.csv')

In [38]:
len(ct_ner['NER-1'])

39020

In [39]:
for k in range(len(ct_ner['NER-1'])):
  list=[]
  for i in ct_ner['NER-1'][k]:
    for j in range(len(ct_mapping['City'])):
      if(i==ct_mapping['City'][j]):
        list.append(ct_mapping['Mapping'][j])
  ct_ner['NER-1'][k]=list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [40]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,52552983299_10157916498178300,Facebook,2020-05-31 23:59:56,2021-08-31 12:54:55,link,George Floyd Death Protesters Return To Capito...,denver.cbslocal.com,"Smart: ""We stand for the truth. We stand for j...",Protesters returned to the Colorado State Capi...,1.0,https://denver.cbslocal.com/2020/05/31/george-...,https://www.facebook.com/52552983299/posts/101...,502230,1.463158,1.0,34,7,41,15,2,2,7,31,0,0,19,8,22,12,5,7,9,13,0,4,20339,CBS Denver,CBSDenver,https://scontent-den4-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Covering Colorado First. Your #1 source for br...,True,2009-02-25 23:34:30,en,0,20339|10157916498178300,[Denver],[],[Denver],[denver],"[Denver, Denver County, Colorado, United States]"
1,1,57427307078_10159355042537079,Facebook,2020-05-31 23:59:53,2021-08-31 20:39:39,link,Psychology professor explains how race factore...,abc7news.com,"Smart: ""We stand for the truth. We stand for j...",What started out as a mostly peaceful demonstr...,1.0,https://abc7news.com/6222900/?ex_cid=TA_KGO_FB...,https://www.facebook.com/57427307078/posts/101...,1219608,-5.451613,1.0,14,5,6,1,0,2,0,3,0,0,50,15,41,9,9,10,21,14,0,6,19637,ABC7 News,abc7news,https://scontent-den4-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Get breaking news and the most talked about st...,True,2009-01-26 18:11:53,en,0,19637|10159355042537079,[Los Angeles],[],[Los Angeles],[los angeles],"[Los Angeles, Los Angeles County, California, ..."
2,2,129617873765147_3588816481178585,Facebook,2020-05-31 23:59:53,2021-06-20 22:23:37,link,Truck hurtles towards protestors in Minneapoli...,express.co.uk,A SHOCKING video has shown a tanker truck driv...,Follow our latest updates on the riots as tens...,1.0,https://www.express.co.uk/news/world/1289602/g...,https://www.facebook.com/129617873765147/posts...,1768821,-3.893333,1.0,17,8,34,0,1,2,2,11,0,0,96,14,105,15,5,29,8,20,0,6,89685,Daily Express,DailyExpress,https://scontent-den4-1.xx.fbcdn.net/v/t1.6435...,facebook_page,GB,The Official Facebook page of the Daily and Su...,True,2010-12-10 10:07:29,en,0,89685|3588816481178585,[],"[Minneapolis, US]",[Minneapolis],"[minneapolis, us]","[Minneapolis, Hennepin County, Minnesota, 5541..."
3,3,821532787927852_3008252662589176,Facebook,2020-05-31 23:59:52,2021-07-09 03:56:24,native_video,Truck hurtles towards protestors in Minneapoli...,express.co.uk,A SHOCKING video has shown a tanker truck driv...,Are You a Uniter or a Divider - Bible lesson T...,21.0,https://www.facebook.com/GunGuyTV/videos/25336...,https://www.facebook.com/821532787927852/posts...,4813,-2.000000,2.0,11,3,0,0,0,0,0,0,0,0,10,5,2,4,1,2,1,3,0,1,11480437,Gunguytv,GunGuyTV,https://scontent-den4-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Find out more about the GunGuy at https://joel...,False,2015-05-01 05:50:26,en,0,11480437|3008252662589176,[],"[Minneapolis, US]",[Minneapolis],"[minneapolis, us]","[Minneapolis, Hennepin County, Minnesota, 5541..."
4,4,123264117724844_3292223030828921,Facebook,2020-05-31 23:59:49,2021-06-27 10:36:41,photo,Truck hurtles towards protestors in Minneapoli...,express.co.uk,A SHOCKING video has shown a tanker truck driv...,Trumbull officials issue statement on George F...,2.0,https://www.facebook.com/TrumbullTimes/photos/...,https://www.facebook.c

In [41]:
ct_ner['NER-1'][12]

['Sacramento, Sacramento County, California, 95814, United States']

In [42]:
ct_ner.to_csv('dataframe0531_post_remove.csv')